In [22]:
import pandas as pd
from campitos import campos, errores
from create_tables import *
import csv
import json

def csv_to_fixture(csv_path, model_name):
    with open(csv_path, 'r') as f:
        reader = csv.DictReader(f)
        fixtures = []
        for row in reader:
            fixture = {
                'model': model_name,
                'pk': int(row['pk']),  # Asume que 'pk' es una columna en tu CSV
                'fields': {field: value for field, value in row.items() if field != 'pk'}
            }
            fixtures.append(fixture)
    return fixtures

def merge_fixtures(fixtures, output_path):
    """
    Merge a list of fixture dictionaries into a single fixture file.

    Args:
        fixtures (list): A list of fixture dictionaries.
        output_path (str or Path): The path to the output file.

    Returns:
        None
    """
    merged = []
    for fixture in fixtures:
        merged.extend(fixture)
    with open(output_path, 'w') as f:
        json.dump(merged, f, indent=4)

def full_sheet(pumps, zones, horizons):
    sheet = pumps.merge(zones, on=['pump','field'], how='inner').merge(horizons, on=['field','pump','zone'], how='inner')
    sheet = sheet[['field','pk_x','pump','pk_y','zone','pk','h_level']]
    sheet = sheet.rename(columns={'pk_x': 'pump_id', 'pk_y': 'zone_id', 'pk':'horizon_id'})
    return sheet

def etl_loop_error(campos):
    for i, field in enumerate(campos):
        print(f'working on {field}')
        id_db = campos[field]['id']
        id = campos[field]['spreadsheet_id']
        if i == 0:
            fields = fields_table(id, id_db)
            fields['name'] = field 
            z, p = zone_and_pump_tables(id, id_db)
            h = horizon_table(id, id_db)
        else:
            fields2 = fields_table(id, id_db)
            fields2['name'] = field
            fields = pd.concat([fields,fields2]).reset_index(drop=True)
            z2, p2 = zone_and_pump_tables(id, id_db)
            z = pd.concat([z, z2]).reset_index(drop=True)
            p = pd.concat([p, p2]).reset_index(drop=True)
            h2 = horizon_table(id, id_db)
            h = pd.concat([h, h2]).reset_index(drop=True)
    
    return fields, p, z, h

def etl_loop(campos):
    for i, field in enumerate(campos):
        print(f'working on {field}')
        try:
            
            id_db = campos[field]['id']
            id = campos[field]['spreadsheet_id']
            if i == 0:
                fields = fields_table(id, id_db)
                fields['name'] = field 
                z, p = zone_and_pump_tables(id, id_db)
                h = horizon_table(id, id_db)
            else:
                fields2 = fields_table(id, id_db)
                fields2['name'] = field
                fields = pd.concat([fields,fields2]).reset_index(drop=True)
                z2, p2 = zone_and_pump_tables(id, id_db)
                z = pd.concat([z, z2]).reset_index(drop=True)
                p = pd.concat([p, p2]).reset_index(drop=True)
                h2 = horizon_table(id, id_db)
                h = pd.concat([h, h2]).reset_index(drop=True)
        except Exception:
            print('error on field')

    return fields, p, z, h

In [23]:
fields, p, z, h = etl_loop(campos)

working on Bellavista 1
working on Bellavista 2
working on Culenar
working on Colin
working on El Sauce
working on La Capilla
working on Las Delicias
error on field
working on Longavi 1
working on Longavi 2
working on Longavi 3
working on Longavi 4
working on Las Liras
working on Las Nueces
working on Liguana
working on Los Maitenes
working on Morza
working on Naguilan
working on Odessa
working on Peñaflor 1
working on Peñaflor 2
working on Piemonte
working on Pullami Sauce
working on Pullami Aromo
working on Pullami Candelaria
working on Pullami Bulnes
working on Quillayes
working on Rauco
working on Rancho
working on San Agustin
working on San Jose 1
working on San Jose 2
working on San Jose 3
working on San Ramon 1
working on San Ramon 2
working on San Ramon 3
working on San Juan
working on Santa Magdalena
working on Teno
working on Torca 1
working on Torca 2
working on Trinidad
working on Vitacura


In [24]:
basics = fields[['basic_field', 'name']]
basics = basics.rename(columns={'basic_field':'pk'})

fields= fields.drop(columns='name')
fields = fields.reset_index().rename(columns={'index':'pk'})
fields['pk'] = fields['pk'] +1

p = p.reset_index().rename(columns={'index':'pk'})
p['pk'] = p['pk'] +1

h = h.reset_index().rename(columns={'index':'pk'})
h['pk'] = h['pk'] +1

z = z.reset_index().rename(columns={'index':'pk'})
z['pk'] = z['pk'] +1


cols_zone = ['rendimiento ton','rendimiento/ha','Nombre','ADP','superficie_sector.1','Cuartel','Rendimiento/ ha','variedad','tesis','Aforo_1']
for col in cols_zone:
    if col in z.columns:
        z = z.drop(columns=[col])


fields.to_csv('./csvs/fields.csv', index=False)
basics.to_csv('./csvs/b.csv',index=False)

In [25]:
h_map = full_sheet(p,z,h)

z_map = h_map.groupby(['field','pump','zone']).first().reset_index().sort_values(['pump_id', 'zone_id'])
z_map = z_map[['field','pump','pump_id','zone','zone_id']]

p_map = z_map.groupby(['field','pump_id']).first().reset_index().sort_values(['pump_id'])
p_map = p_map[['field','pump','pump_id']]

In [26]:
zp = z.merge(z_map, on=['pump','zone','field']).drop(columns=['pump', 'zone_id']).rename(columns={'pump_id':'pump', 'zone':'zone_number'})
hp = h.merge(h_map, on=['pump','zone','field', 'h_level']).drop(columns=['pump','zone','horizon_id','pump_id', 'field']).rename(columns={'zone_id':'zone'})

In [27]:
p = p.rename(columns={'pump':'name'})
p.to_csv('./csvs/p.csv', index=False)
zp.to_csv('./csvs/z.csv', index=False)
hp.to_csv('./csvs/h.csv', index=False)

In [28]:
fixtures = [
    csv_to_fixture('./csvs/b.csv', 'gota_app.basicfield'),
    csv_to_fixture('./csvs/fields.csv', 'gota_app.seasonfield'),
    csv_to_fixture('./csvs/p.csv', 'gota_app.pump'),
    csv_to_fixture('./csvs/z.csv', 'gota_app.zone'),
    csv_to_fixture('./csvs/h.csv', 'gota_app.horizon'),
]

In [29]:
merge_fixtures(fixtures, 'data.json')

In [8]:

fields, p, z, h = etl_loop_error(errores)

working on Pullami Sauce
working on Los Maitenes


In [9]:
fields

INFORMACION,weather_factor,rain_factor,dropcontrol_id,day_starttime,delay,weekday,weather_station,season,basic_field,name
0,0,0.6,1019,08:00,10,1,1,1,23,Pullami Sauce
1,0,0.6,479,8:00,15,4,1,1,15,Los Maitenes
